# Testing LDF for grouping Paulis

compiled by Rudy Raymond / IBM Research -- Tokyo on May 20, 2020

In [1]:
from qiskit.aqua.operators.pauli_graph import PauliGraph
from qiskit.quantum_info.operators.pauli import Pauli
import warnings
warnings.filterwarnings('ignore')            #surpressing deprecation warning

## Functions to group Paulis

In [2]:
from ast import literal_eval 

def read_dict(filename):
    f = open(filename, "r")
    d = literal_eval(f.read())
    #print("RD", d)
    answers = []
    for ad in d["paulis"]:
        answers.append((ad["coeff"]["real"], ad["label"]))
    return answers
    
def read_inputfile(filename):
    with open(filename, "r") as f:
        lineno = 0
        answers = []
        for line in f:
            #print(line, line.startswith == "\{")
            if lineno == 0 and line.startswith("{"):
                #this is dictionary of Pauli, so read the file into dictionary
                #print("detecting dict file")
                f.close()
                answers = read_dict(filename)
                return answers
                
            if lineno % 2 == 0:
                pLabel = line.rstrip()
            else:
                weight = complex(line.rstrip().replace("(","").replace(")", ""))
                weight = weight.real #comment out this if complex value is allowed
                answers.append((weight, pLabel))
            lineno += 1
    return answers

def ldf_group_outfile(infilename, outfilename=None):
    if outfilename is None:
        outfilename = infilename.replace("txt", "grouped.txt")
    pList = read_inputfile(infilename)
    #print(pList)
    pauliList = [  (each[0], Pauli(label=each[1])) for each in pList  ]
    pauliGraph = PauliGraph(pauliList)
    groupedPaulis = pauliGraph.grouped_paulis
    with open(outfilename, "w") as f:
        for group in groupedPaulis:
            for each in group:
                f.write(str(each[0])+","+str(each[1])+"\n")

## List of files with Pauli lists to group

In [3]:
infiles = ("H2_6-31G_8qubits/H .0 .0 .0 ; H 0 0 .7462 jordan_wigner_Pauli_list.txt",\
"H2_6-31G_8qubits/H .0 .0 .0 ; H 0 0 .7462 bravyi_kitaev_Pauli_list.txt",\
"H2_6-31G_8qubits/H .0 .0 .0 ; H 0 0 .7462 parity_Pauli_list.txt",\
"LiH_STO3g_12qubits/H .0 .0 .0 ; Li .0 .0 1.548parity_Pauli_list.txt",\
"LiH_STO3g_12qubits/H .0 .0 .0 ; Li .0 .0 1.548jordan_wigner_Pauli_list.txt",\
"LiH_STO3g_12qubits/H .0 .0 .0 ; Li .0 .0 1.548bravyi_kitaev_Pauli_list.txt",\
#"H2O_ccpvdz_48qubits/H .0 0.750 -0.487 ; H .0 -0.750 -0.487 ; O 0.0 0.0 0.122 jordan_wigner_Pauli_list.txt",\
#"H2O_ccpvdz_48qubits/H .0 0.750 -0.487 ; H .0 -0.750 -0.487 ; O 0.0 0.0 0.122 bravyi_kitaev_Pauli_list.txt",\
#"H2O_ccpvdz_48qubits/H .0 0.750 -0.487 ; H .0 -0.750 -0.487 ; O 0.0 0.0 0.122 parity_Pauli_list.txt",\
"NH3_STO3g_16qubits/N .0 .0 0.149 ; H .0 0.947 -0.349 ; H 0.820 -0.474 -0.349 ; H -0.820 -0.474 -0.349 bravyi_kitaev_Pauli_list.txt",\
"NH3_STO3g_16qubits/N .0 .0 0.149 ; H .0 0.947 -0.349 ; H 0.820 -0.474 -0.349 ; H -0.820 -0.474 -0.349 jordan_wigner_Pauli_list.txt",\
"NH3_STO3g_16qubits/N .0 .0 0.149 ; H .0 0.947 -0.349 ; H 0.820 -0.474 -0.349 ; H -0.820 -0.474 -0.349 parity_Pauli_list.txt",\
"HCl_STO3g_20qubits/Cl .0 .0 .075; H .0 .0 -1.268 bravyi_kitaev_Pauli_list.txt",\
"HCl_STO3g_20qubits/Cl .0 .0 .075; H .0 .0 -1.268 jordan_wigner_Pauli_list.txt",\
"HCl_STO3g_20qubits/Cl .0 .0 .075; H .0 .0 -1.268 parity_Pauli_list.txt",\
"H2O_STO3g_14qubits/H .0 0.769 -0.546 ; O .0 .0 0.137 ; H .0 -0.769 -0.546bravyi_kitaev_Pauli_list.txt",\
"H2O_STO3g_14qubits/H .0 0.769 -0.546 ; O .0 .0 0.137 ; H .0 -0.769 -0.546parity_Pauli_list.txt",\
"H2O_STO3g_14qubits/H .0 0.769 -0.546 ; O .0 .0 0.137 ; H .0 -0.769 -0.546jordan_wigner_Pauli_list.txt",\
"H2O_6-31G_26qubits/H .0 0.795 -0.454 ; H .0 -0.795 -0.454 ; O 0.0 0.0 0.113 parity_Pauli_list.txt",\
"H2O_6-31G_26qubits/H .0 0.795 -0.454 ; H .0 -0.795 -0.454 ; O 0.0 0.0 0.113 jordan_wigner_Pauli_list.txt",\
"H2O_6-31G_26qubits/H .0 0.795 -0.454 ; H .0 -0.795 -0.454 ; O 0.0 0.0 0.113 bravyi_kitaev_Pauli_list.txt",\
"BeH2_STO3g_14qubits/H 1.3038 .0 .0 ; Be .0 .0 .0 ; H -1.3038 .0 .0bravyi_kitaev_Pauli_list.txt",\
"BeH2_STO3g_14qubits/H 1.3038 .0 .0 ; Be .0 .0 .0 ; H -1.3038 .0 .0parity_Pauli_list.txt",\
"BeH2_STO3g_14qubits/H 1.3038 .0 .0 ; Be .0 .0 .0 ; H -1.3038 .0 .0jordan_wigner_Pauli_list.txt",\
"C2_STO3g_20qubits/C .0 .0 .0 ; C 0 0 1.2691 bravyi_kitaev_Pauli_list.txt",\
"C2_STO3g_20qubits/C .0 .0 .0 ; C 0 0 1.2691 jordan_wigner_Pauli_list.txt",\
"C2_STO3g_20qubits/C .0 .0 .0 ; C 0 0 1.2691 parity_Pauli_list.txt",\
#"H2O_6-31G*_36qubits/H .0 0.764 -0.479 ; H .0 -0.764 -0.479 ; O 0.0 0.0 0.120 jordan_wigner_Pauli_list.txt",\
#"H2O_6-31G*_36qubits/H .0 0.764 -0.479 ; H .0 -0.764 -0.479 ; O 0.0 0.0 0.120 parity_Pauli_list.txt",\
#"H2O_6-31G*_36qubits/H .0 0.764 -0.479 ; H .0 -0.764 -0.479 ; O 0.0 0.0 0.120 bravyi_kitaev_Pauli_list.txt",\
"H2_STO3g_4qubits/H .0 .0 .0 ; H .0 .0 .735jordan_wigner_Pauli_list.txt",\
"H2_STO3g_4qubits/H .0 .0 .0 ; H .0 .0 .735bravyi_kitaev_Pauli_list.txt",\
"H2_STO3g_4qubits/H .0 .0 .0 ; H .0 .0 .735parity_Pauli_list.txt")

In [4]:
for infile in infiles:
    print("Processing:", infile)
    ldf_group_outfile(infile)

Processing: H2_6-31G_8qubits/H .0 .0 .0 ; H 0 0 .7462 jordan_wigner_Pauli_list.txt
Processing: H2_6-31G_8qubits/H .0 .0 .0 ; H 0 0 .7462 bravyi_kitaev_Pauli_list.txt
Processing: H2_6-31G_8qubits/H .0 .0 .0 ; H 0 0 .7462 parity_Pauli_list.txt
Processing: LiH_STO3g_12qubits/H .0 .0 .0 ; Li .0 .0 1.548parity_Pauli_list.txt
Processing: LiH_STO3g_12qubits/H .0 .0 .0 ; Li .0 .0 1.548jordan_wigner_Pauli_list.txt
Processing: LiH_STO3g_12qubits/H .0 .0 .0 ; Li .0 .0 1.548bravyi_kitaev_Pauli_list.txt
Processing: NH3_STO3g_16qubits/N .0 .0 0.149 ; H .0 0.947 -0.349 ; H 0.820 -0.474 -0.349 ; H -0.820 -0.474 -0.349 bravyi_kitaev_Pauli_list.txt
Processing: NH3_STO3g_16qubits/N .0 .0 0.149 ; H .0 0.947 -0.349 ; H 0.820 -0.474 -0.349 ; H -0.820 -0.474 -0.349 jordan_wigner_Pauli_list.txt
Processing: NH3_STO3g_16qubits/N .0 .0 0.149 ; H .0 0.947 -0.349 ; H 0.820 -0.474 -0.349 ; H -0.820 -0.474 -0.349 parity_Pauli_list.txt
Processing: HCl_STO3g_20qubits/Cl .0 .0 .075; H .0 .0 -1.268 bravyi_kitaev_Pauli_l

## Testing lines

In [ ]:
answers = read_inputfile("BeH2_STO3g_14qubits/H 1.3038 .0 .0 ; Be .0 .0 .0 ; H -1.3038 .0 .0bravyi_kitaev_Pauli_list.txt")

In [ ]:
for each in answers:
    print(each, len(each[1]))

In [ ]:
paulis = [ (each[0], Pauli(label=each[1])) for each in answers]

In [ ]:
paulis[1]

In [ ]:
answers[1][1]

In [ ]:
pg = PauliGraph(paulis)

In [ ]:
gp = pg.grouped_paulis

In [ ]:
gp

In [ ]:
for each in gp[-1]:
    print(each[0], each[1])

In [ ]:
len(answers)